## Pre-collect all the data in this cell

In [54]:
import os
from typing import Optional

import requests
from dotenv import load_dotenv
from gql import Client
from gql.transport.requests import RequestsHTTPTransport
from web3 import Web3

load_dotenv()

ARB_CHAIN_ID = 42161
BALANCER_GAUGE_URL = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-gauges"
BALANCER_GAUGE_CONTROLLER_ADDR = Web3.toChecksumAddress("0xC128468b7Ce63eA702C1f104D55A2566b13D3ABD")
BALANCER_GAUGE_CONTROLLER_ABI = [{"stateMutability": "view", "type": "function", "name": "gauge_relative_weight",
                                  "inputs": [{"name": "addr", "type": "address"}],
                                  "outputs": [{"name": "", "type": "uint256"}]},
                                 {"stateMutability": "view", "type": "function", "name": "gauge_relative_weight",
                                  "inputs": [{"name": "addr", "type": "address"}, {"name": "time", "type": "uint256"}],
                                  "outputs": [{"name": "", "type": "uint256"}]}]

web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))


def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


client = make_gql_client(BALANCER_GAUGE_URL)

# Fetch all voting gauges from github json
voting_gauges_req = requests.get(
    "https://raw.githubusercontent.com/balancer/frontend-v2/master/src/data/voting-gauges.json")
if not voting_gauges_req.ok:
    raise ValueError("Failed to fetch voting gauges")
voting_gauges = voting_gauges_req.json()

# Collect arb gauges
arb_gauges = {}
for gauge in voting_gauges:
    # Only collect gauges for the arb chain and that are not killed
    if int(gauge['network']) == ARB_CHAIN_ID and gauge['isKilled'] is False:
        arb_gauges[gauge['address']] = gauge

gauge_c_contract = web3.eth.contract(address=BALANCER_GAUGE_CONTROLLER_ADDR, abi=BALANCER_GAUGE_CONTROLLER_ABI)

vote_weights = {}
# Collect gauge voting weights from the gauge controller on chain
for gauge_addr, gauge_data in arb_gauges.items():
    weight = gauge_c_contract.functions.gauge_relative_weight(gauge_addr).call() / 1e18 * 100
    vote_weights[gauge_addr] = weight
    arb_gauges[gauge_addr]['voteWeight'] = weight

## Print gauges sorted by vote weight

In [55]:
from copy import deepcopy
from IPython.core.display import HTML
import pandas as pd

arb_gauges_copy = deepcopy(arb_gauges)
# Remove some fields from the gauges
for gauge in arb_gauges_copy.values():
    gauge.pop('tokenLogoURIs', None)
    symbol = gauge['pool']['symbol']
    pool_addr = gauge['pool']['address']
    pool_id = gauge['pool']['id']
    gauge['pool_symbol'] = symbol
    gauge['pool_addr'] = pool_addr
    gauge['pool_id'] = pool_id
    gauge.pop('pool', None)
    gauge.pop('network', None)
    gauge.pop('isKilled', None)
    gauge.pop('relativeWeightCap', None)

arb_gauges_df = pd.DataFrame.from_dict(arb_gauges_copy, orient='index')
arb_gauges_df = arb_gauges_df.sort_values(by='voteWeight', ascending=False)
display(HTML(arb_gauges_df.to_html(index=False)))

address,addedTimestamp,voteWeight,pool_symbol,pool_addr,pool_id
0xDf464348c4EC2Bf0e5D6926b9f707c8e02301adf,1683715019,0.937519,B-wstETH-WETH-Stable,0x36bf227d6BaC96e2aB1EbB5492ECec69C691943f,0x36bf227d6bac96e2ab1ebb5492ecec69c691943f000200000000000000000316
0x8135d6AbFd42707A87A7b94c5CFA3529f9b432AD,1683715019,0.821489,RDNT-WETH,0x32dF62dc3aEd2cD6224193052Ce665DC18165841,0x32df62dc3aed2cd6224193052ce665dc181658410002000000000000000003bd
0x175407b4710b5A1cB67a37C76859F17fb2ff6672,1687806647,0.777957,55auraBal-45wsteth,0xc7FA3A3527435720f0e2a4c1378335324dd4F9b3,0xc7fa3a3527435720f0e2a4c1378335324dd4f9b3000200000000000000000459
0xa8D974288Fe44ACC329D7d7a179707D27Ec4dd1c,1684791827,0.474528,50tBTC-50WETH,0xc9f52540976385A84BF416903e1Ca3983c539E34,0xc9f52540976385a84bf416903e1ca3983c539e34000200000000000000000434
0x4944b07977A42C15c6a06CF4e204e24c60564104,1683715019,0.454235,rETH-bb-a-WETH-BPT,0xCba9Ff45cfB9cE238AfDE32b0148Eb82CbE63562,0xcba9ff45cfb9ce238afde32b0148eb82cbe635620000000000000000000003fd
0xc4b6cc9A444337b1Cb8cBbDD9de4d983f609C391,1684791827,0.233378,2BTC,0x542F16DA0efB162D20bF4358EfA095B70A100f9E,0x542f16da0efb162d20bf4358efa095b70a100f9e000000000000000000000436
0x25869f277f474FA9459F40F5D4cB0D6A8Ab41967,1687258571,0.210626,DOLA/USDC BPT,0x8bc65Eed474D1A00555825c91FeAb6A8255C2107,0x8bc65eed474d1a00555825c91feab6a8255c2107000000000000000000000453
0xacE0D479040231e3c6b17479cFd4444182d521d4,1687258571,0.210626,DOLA/bbaUSD-BPT,0x567ECfCB22205D279BB8Eed3E066989902bF03D5,0x567ecfcb22205d279bb8eed3e066989902bf03d5000000000000000000000452
0x0EDF6cDd81BC3471C053341B7D8Dfd1Cb367AD93,1683715019,0.183482,50MAGIC-50USDC,0xb3028Ca124B80CFE6E9CA57B70eF2F0CCC41eBd4,0xb3028ca124b80cfe6e9ca57b70ef2f0ccc41ebd40002000000000000000000ba
0xd758454BDF4Df7Ad85f7538DC9742648EF8e6d0A,1686659711,0.091146,80PAL-20OHM,0x8d333f82e0693f53fA48c40d5D4547142E907e1D,0x8d333f82e0693f53fa48c40d5d4547142e907e1d000200000000000000000437
